In [1]:
import os
import uuid

import pandas as pd
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, PointStruct, VectorParams
from sentence_transformers import SentenceTransformer

load_dotenv()

%load_ext autoreload
%autoreload 2

c:\Users\S\PycharmProjects\SimilarBooksRecommendation\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
df_1 = pd.read_csv(os.path.join(os.getcwd(), "data", "комиксы.csv"), on_bad_lines="skip").dropna()
df_2 = pd.read_csv(os.path.join(os.getcwd(), "data", "средневековье.csv"), on_bad_lines="skip").dropna()
df_3 = pd.read_csv(os.path.join(os.getcwd(), "data", "художественная.csv"), on_bad_lines="skip").dropna()

df = pd.concat([df_1, df_2, df_3], axis=0).drop_duplicates(subset=["Title", "Author"]).drop_duplicates(subset=["Title"])
df = df[df["Description"].apply(len) >= 100]

df["ID"] = [uuid.uuid5(uuid.NAMESPACE_DNS, title).hex for title in df["Title"]]
df['Title'] = df['Title'].apply(lambda x: x.lower().capitalize())

In [3]:
print(df.shape)
display(df.head())

(16149, 8)


,Title,Author,Link,Image,Category,Description,Info,ID
0,Истории книжных магазинов,Ивашкина М.,https://www.podpisnie.ru/books/istorii-knizhny...,/upload/resize_cache/iblock/cfe/160_230_1/8vg4...,Графические романы / Комиксы,Книжный магазин — место где книга и человек на...,Автор Ивашкина М. Издательство Миля Год издани...,e26498c9c4a75252a18a1c677b6d3ae9
1,Полуночники,Эвенс Б.,https://www.podpisnie.ru/books/polunochniki/,/upload/resize_cache/iblock/a5b/160_230_1/tj5v...,Графические романы / Комиксы,Одна ночь. Три незнакомца. Три совершенно разн...,Автор Эвенс Б. Издательство Бумкнига Год издан...,2d4a036599e959a0a9e76e32a2075200
2,Госпожа кагуя: в любви как на войне. любовная...,Акасака А.,https://www.podpisnie.ru/books/gospozha-kaguya...,/upload/resize_cache/iblock/7fd/160_230_1/uqfn...,Графические романы / Комиксы,Завещание Ганъана Синомии уничтожено и Кагуя в...,Автор Акасака А. Издательство Азбука Год издан...,9ac06e3bafd25975bbf6fb9083e18567
3,"Люди, которые легко становятся счастливыми. ...",Дэнсинг С.,https://www.podpisnie.ru/books/lyudi-kotorye-l...,/upload/resize_cache/iblock/929/160_230_1/y4yj...,Графические романы / Комиксы,Привычная рутина отбирает краски жизни а о «ма...,Автор Дэнсинг С. Издательство КоЛибри Год изда...,ca68d65336425ee49bb1e563fb832e1f
4,"Моя геройская академия. кн. 19. те, кто об...",Хорикоси К.,https://www.podpisnie.ru/books/moya-geroyskaya...,/upload/resize_cache/iblock/f6d/160_230_1/7hh3...,Графические романы / Комиксы,Решающее сражение за судьбу мира в самом разга...,Автор Хорикоси К. Издательство Азбука Год изда...,2cb378c590605ddb8c4cf85b8ee1518f


In [4]:
# import joblib
# hash_map = {uid: title for uid, title in zip(df["ID"], df["Title"])}
# joblib.dump(hash_map, os.path.join(os.getcwd(), "backend", "config", "title_id_hash_map.pkl"))

In [4]:
embedder = SentenceTransformer(os.getenv("QDRANT_EMBEDDER"))
qdrant = QdrantClient(url=os.getenv("QDRANT_URL"), api_key=os.getenv("QDRANT_API_KEY"))

In [ ]:
# vectors = embedder.encode(df["Description"].values, batch_size=1, normalize_embeddings=True, show_progress_bar=True)

In [ ]:
# qdrant.recreate_collection(
#     collection_name=os.getenv("QDRANT_COLLECTION_NAME"),
#     vectors_config=VectorParams(size=312, distance=Distance.COSINE),
# )

In [ ]:
# from tqdm import tqdm

# # Параметры для пакетной обработки
# batch_size = 300  # Размер пакета, можно настроить по мере необходимости
# points = []

# # Обрабатываем данные по пакетам
# for uid, category, author, title, image_link, vector in tqdm(
#     zip(df["ID"], df["Category"], df["Author"], df["Title"], df["Image"], vectors)
# ):
#     points.append(
#         PointStruct(
#             id=uid,
#             payload={
#                 "category": category,
#                 "author": author,
#                 "title": title,
#                 "image_link": image_link,
#             },
#             vector=vector,
#         )
#     )

#     # Если пакет набран, отправляем его в Qdrant
#     if len(points) >= batch_size:
#         qdrant.upsert(collection_name=os.getenv("QDRANT_COLLECTION_NAME"), points=points)
#         points = []  # Очищаем список для следующего пакета

# # Отправляем оставшиеся точки, если они есть
# if points:
#     qdrant.upsert(collection_name=os.getenv("QDRANT_COLLECTION_NAME"), points=points)

In [5]:
query_vector = embedder.encode("Ведьмак Предназначение")
n_chunks = 5
offset = 0

In [7]:
content = qdrant.search(
    collection_name=os.getenv("QDRANT_COLLECTION_DESCRIPTION"), query_vector=query_vector, limit=n_chunks, offset=offset
)
content

[ScoredPoint(id='7654c1a4-09e6-5ff8-817d-b43929fa6d87', version=39, score=0.8305099, payload={'author': 'Сапковский А.', 'category': 'Художественная литература', 'image_link': '/upload/resize_cache/iblock/fb9/160_230_1/6tjf0vcl6qf92zgz17ku1t1ewba720o8.jpg', 'title': 'Ведьмак.  Меч Предназначения'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='92444711-200a-5c29-aa4a-892effbf7a05', version=41, score=0.82693565, payload={'author': 'Сапковский А.', 'category': 'Художественная литература', 'image_link': '/upload/resize_cache/iblock/c53/160_230_1/tigq1chxqc0e3yev85p31n39xaly6h3f.jpg', 'title': 'Ведьмак.  Последнее желание'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='de468708-c02d-5054-b28c-d3284c5bd3c5', version=40, score=0.8225369, payload={'author': 'Сапковский А.', 'category': 'Художественная литература', 'image_link': '/upload/resize_cache/iblock/c71/160_230_1/e24vw9bjg1l753zw24w2du49pzh9e86q.jpg', 'title': 'Ведьмак.  Час Презрения (2изд)'}, ve

# Redis

In [1]:
import os

import redis

# Подключение к Redis
redis_client = redis.StrictRedis(
    host=os.getenv("REDIS_HOST"), port=os.getenv("REDIS_PORT"), password=os.getenv("RESID_PASSWORD")
)

try:
    # Проверка подключения к Redis
    pong = redis_client.ping()
    if pong:
        print("Подключение к Redis успешно!")
    else:
        print("Не удалось подключиться к Redis.")
except redis.ConnectionError as e:
    print(f"Ошибка подключения к Redis: {e}")

Подключение к Redis успешно!


In [5]:
# from backend.src.scripts.redis_scripts import get_book_info_redis


# get_book_info_redis("Ведьмак", redis_client)

## Быстрая загрузка данных в redis

In [9]:
# Очистка всех данных
redis_client.flushall()

True

In [10]:
from tqdm import tqdm

# Преобразование данных в список кортежей для удобства использования
data = list(zip(df["Title"], df["Description"], df["Category"], df["Author"], df["Image"], df["Info"], df["ID"]))
# Определение размера батча
batch_size = 1000

# Использование пайплайна для записи данных и метаданных
for i in tqdm(range(0, len(data), batch_size)):
    with redis_client.pipeline() as pipe:
        for title, description, category, author, image, info, uid in data[i:i + batch_size]:
            # Установка данных
            pipe.set(title, description)

            # Установка метаданных
            metadata_key = f"{title}:metadata"
            pipe.hset(metadata_key, "category", category)
            pipe.hset(metadata_key, "author", author)
            pipe.hset(metadata_key, "image", image)
            pipe.hset(metadata_key, "info", info)
            pipe.hset(metadata_key, "uid", uid)

        # Выполнение всех команд в пайплайне
        pipe.execute()

100%|██████████| 17/17 [00:04<00:00,  3.46it/s]


In [11]:
num_keys = redis_client.dbsize()

print(f"Number of keys in the database: {num_keys}")

Number of keys in the database: 32146


# Поиск описания книги по ее названию

In [ ]:
query = "Полуночники"

In [6]:
qdrant.recreate_collection(
    collection_name="BookTitles",
    vectors_config=VectorParams(size=312, distance=Distance.DOT),
)

C:\Users\S\AppData\Local\Temp\ipykernel_17928\204341076.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [7]:
vectors = embedder.encode(df["Title"].values, batch_size=16, normalize_embeddings=True, show_progress_bar=True)

Batches:   0%|          | 0/1010 [00:00<?, ?it/s]

In [8]:
from tqdm import tqdm

# Параметры для пакетной обработки
batch_size = 300  # Размер пакета, можно настроить по мере необходимости
points = []

# Обрабатываем данные по пакетам
for uid, title, vector in tqdm(zip(df["ID"], df["Title"], vectors)):
    points.append(
        PointStruct(
            id=uid,
            payload={
                "title": title,
            },
            vector=vector,
        )
    )

    # Если пакет набран, отправляем его в Qdrant
    if len(points) >= batch_size:
        qdrant.upsert(collection_name="BookTitles", points=points)
        points = []  # Очищаем список для следующего пакета

# Отправляем оставшиеся точки, если они есть
if points:
    qdrant.upsert(collection_name="BookTitles", points=points)

16149it [00:53, 300.10it/s]


In [9]:
from qdrant_client.http import models


def filter_search(query, collection_name, limit=6, offset=0, title_filter=None):
    # Создание векторного запроса
    query_vector = embedder.encode(
        query,
        batch_size=1,
        normalize_embeddings=True,
    )

    # Построение условий фильтрации
    filter_conditions = []
    if title_filter:
        filter_conditions.append(models.FieldCondition(key="title", match=models.MatchValue(value=title_filter)))

    # Выполнение поиска
    results = qdrant.search(
        collection_name=collection_name,
        query_vector=query_vector,
        query_filter=models.Filter(must=filter_conditions) if filter_conditions else None,
        limit=limit,
        offset=offset,
    )

    return results

In [10]:
n_chunks = 20
offset = 0
query = "Ведьмак"
query_vector = embedder.encode(query)

content = qdrant.search(collection_name="BookTitles", query_vector=query_vector, limit=n_chunks, offset=offset)
content

[ScoredPoint(id='e22499a5-3308-5e07-a417-a253e46b5988', version=24, score=0.99999994, payload={'title': 'Ведьмак'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='252031e8-29d3-5b10-8abc-6126c01104ef', version=8, score=0.9540581, payload={'title': 'Ведьма'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='273b8a09-f860-5e76-8b68-f744c7fe055d', version=15, score=0.92493194, payload={'title': 'Ведьмочервь'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='7cf50021-2a19-540b-94c4-6390767cccb9', version=48, score=0.9165741, payload={'title': 'Войцек'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='7654c1a4-09e6-5ff8-817d-b43929fa6d87', version=39, score=0.9118993, payload={'title': 'Ведьмак.  Меч Предназначения'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='6d90600a-7b7d-596c-8171-8a29098dd54b', version=30, score=0.9106577, payload={'title': 'Дедейме'}, vector=None, shard_key=None, order_value=None)

In [14]:
# Пример запроса
search_results = filter_search(
    query=query,
    collection_name="BookTitles",
    limit=n_chunks,
    # category_filter=category,
    # author_filter=author,
    title_filter=query,
)

for result in search_results:
    print(f"Title: {result.payload['title']}, Score: {result.score}")

Title: Ведьмак, Score: 1.0


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def ranking_titles(content: PointStruct, query):
    # Преобразуем данные в DataFrame
    df = pd.DataFrame(content, columns=["values"])

    # Объединяем заголовки и запрос для векторизации
    titles = df["values"].tolist()
    titles.append(query)

    # Векторизация текстов с помощью TF-IDF
    tfidf_matrix = TfidfVectorizer().fit_transform(titles)

    # Добавляем косинусное сходство в DataFrame и сортируем по нему
    df["similarity"] = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1]).flatten()
    df = df.sort_values(by="similarity", ascending=False)
    df = df[df["similarity"] != 0.0].iloc[:10]
    # Выводим отсортированный DataFrame
    return df["values"].to_list()

In [ ]:
data = []
for msg in content:
    data.append(msg.payload["title"])

In [ ]:
data

In [ ]:
ranking_titles(data, query)

# Exceptions:

In [ ]:
from backend.src.exceptions.custom_exceptions import RedisDataIsNotFound

In [ ]:
raise RedisDataIsNotFound(RedisDataIsNotFound.default_message + "Aboba")

In [ ]:
import pandas as pd

df = pd.DataFrame([1, 2, 3])

In [ ]:
df.empty

In [20]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False


class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True

    def search_prefix(self, prefix):
        node = self.root
        for char in prefix:
            if char not in node.children:
                return []
            node = node.children[char]
        return self._collect_all_words(node, prefix)

    def _collect_all_words(self, node, prefix):
        words = []
        if node.is_end_of_word:
            words.append(prefix)
        for char, child_node in node.children.items():
            words.extend(self._collect_all_words(child_node, prefix + char))
        return words


# Функция автокомплита
def autocomplete_books_trie(prefix, trie, limit=5):
    suggestions = trie.search_prefix(prefix.lower())
    return suggestions[:limit]


# Инициализация Trie и вставка книг
trie = Trie()

# books = df["Title"]

# for book in books:
#     trie.insert(book.lower())

# import joblib


# joblib.dump(trie,os.path.join(os.getcwd(), "data", "trie.pkl"))

In [3]:
# Пример использования
user_input = "Ведьмак"
suggestions = autocomplete_books_trie(user_input, trie, limit=10)
print(list(map(str.capitalize, suggestions)))

NameError: name 'autocomplete_books_trie' is not defined

In [3]:
from backend.src.scripts.redis_scripts import get_book_info_redis

user_input = "Ведьмак"
get_book_info_redis(user_input, redis_client).metadata

RedisMetadata(category='Художественная литература', author='Сапковский А. Монтень Т.', image='/upload/resize_cache/iblock/28f/160_230_1/axqvzbi66blc5e3xcrau287wqmmzlti5.jpg', info='Автор Сапковский А. Монтень Т. Издательство АСТ Год издания 2021 Переплет Твёрдый Страниц 56 Формат 245х333 мм Язык Русский ISBN 978-5-17-137657-4 Артикул 1146488', uid='e22499a533085e07a417a253e46b5988')

In [1]:
import joblib
import os 
trie = joblib.load(os.path.join(os.getcwd(), "data", "trie.pkl"))

AttributeError: Can't get attribute 'Trie' on <module '__main__'>